In [2]:
from darkflow.net.build import TFNet
import cv2
import numpy as np

read_option=2  #1 read image #2 read camera
options = {"pbLoad": "built_graph/tiny-yolo_arrow.pb",
           "threshold": 0.1,
           "metaLoad": "built_graph/tiny-yolo_arrow.meta" ,
           "gpu": 1.0
           }        
           
tfnet = TFNet(options)




def boxing(original_img, predictions):
    newImage = np.copy(original_img)

    for result in predictions:
        top_x = result['topleft']['x']
        top_y = result['topleft']['y']

        btm_x = result['bottomright']['x']
        btm_y = result['bottomright']['y']

        confidence = result['confidence']
        label = result['label'] + " " + str(round(confidence, 3))

        if confidence > 0.1:
            newImage = cv2.rectangle(newImage, (top_x, top_y), (btm_x, btm_y), (255,0,0), 3)
            newImage = cv2.putText(newImage, label, (top_x, top_y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.8, (0, 230, 0), 1, cv2.LINE_AA)
            
    return newImage



if read_option==1:
    imgcv = cv2.imread("images/frame232.jpg")
    result = tfnet.return_predict(imgcv)
    print(result)

    if len(result)!=0:
        %matplotlib inline
        import matplotlib.pyplot as plt

        _, ax = plt.subplots(figsize=(20, 10))
        ax.imshow(boxing(imgcv, result))
    else:
        print("Nothing detected")
else:
    cap = cv2.VideoCapture(0) # 0 represents the in-built camera (It can also be changed to the name of a video file or other camera)
    print("Start detecting")
    while(True):
        # Capture frame-by-frame
        ret, frame = cap.read() #ret is boolean value (True for frame reading correctly otherwise False)
        result = tfnet.return_predict(frame)
        cv2.imshow('frame',boxing(frame,result)) 
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()



Loading from .pb and .meta
GPU mode with 1.0 usage
Start detecting


AssertionError: Image is not a np.ndarray